# Исследование информации о книгах для стартапа - сервиса для чтения книг по подписке

Исследование информации о книгах, издательствах, авторах, а так же пользовательских обзорах содержащейся в БД с **целью** помощи в **формулировке ценностного предложения** для нового продукта - сервиса для чтения книг по подписке.  

Задачи:  
- подсчет количества книг выпущеных после 1 января 2020;  
- подсчет количества обзоров и средней оценки для каждого издания;
- идентификация издательства, выпустившего наибольшее число книг толще 50 страниц (для исключения брошюр из анализа);
-определение автора с наиболее высокой средней оценкой книг (только для книг с 50 и более оценками);
-подсчет среднего количесва обзоров от пользователей (только для пользователей поставивших более 48 оценок).

Создаем подключение к базе данных

In [220]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [221]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas, проверяем корректность работы подключения
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


## Проверка содержания таблиц

In [222]:
#создадим список для перебора таблиц в цикле
table_list = ['books', 'authors', 'ratings', 'reviews', 'publishers']

In [223]:
#функция для просмотра таблиц
def open_tab(tab):
    print(f'Первые пять строк таблицы {tab}')
    query = '''SELECT * FROM {} LIMIT 5'''.format(tab)
    con=engine.connect()
    display(pd.io.sql.read_sql(sql=text(query), con = con))
    print()

In [225]:
#вызываем в цикле каждую таблицу
for tab in table_list:
    open_tab(tab)

Первые пять строк таблицы books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



Первые пять строк таблицы authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



Первые пять строк таблицы ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



Первые пять строк таблицы reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...



Первые пять строк таблицы publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Ответы на вопросы

**Задача 1:** Cколько книг вышло после 1 января 2000 года

In [226]:
query = '''SELECT COUNT(DISTINCT(book_id))
          FROM books as b
          WHERE b.publication_date > '2000-01-01';'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


**Задача 2:** Для каждой книги посчитайте количество обзоров и среднюю оценку

In [228]:
print('Количество обзоров и средняя оценка для книг')
query = '''SELECT DISTINCT(b.title) as title,
                b.book_id as book_id,
                  COUNT(DISTINCT(rev.review_id)) AS review_cnt,
                  AVG(DISTINCT(rat.rating)) as avg_rating
          FROM books AS b
          FULL OUTER JOIN ratings AS rat ON b.book_id = rat.book_id
          FULL OUTER JOIN reviews AS rev ON b.book_id = rev.book_id
          GROUP BY b.book_id
          ORDER BY COUNT(DISTINCT(rev.review_id)) DESC;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

Количество обзоров и средняя оценка для книг


,title,book_id,review_cnt,avg_rating
0,Twilight (Twilight #1),948,7,3.00
1,Eat Pray Love,207,6,3.00
2,Harry Potter and the Chamber of Secrets (Harry...,299,6,3.00
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,3.50
4,Outlander (Outlander #1),497,6,3.25
...,...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,191,0,4.00
996,Essential Tales and Poems,221,0,4.00
997,Leonardo's Notebooks,387,0,4.00
998,The Cat in the Hat and Other Dr. Seuss Favorites,672,0,5.00


In [229]:
print('Количество уникальных наименований и идентификаторов в таблице books')
query = '''SELECT COUNT(DISTINCT(b.title)) as cnt_unique_title,
                  COUNT(DISTINCT(b.book_id)) as cnt_unique_book_id_in_books
          FROM books AS b;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

Количество уникальных наименований и идентификаторов в таблице books


,cnt_unique_title,cnt_unique_book_id_in_books
0,999,1000


In [230]:
print('Количество уникальных идентификаторов в таблице reviews')
query = '''SELECT COUNT(DISTINCT(rev.book_id)) as cnt_unique_book_id_in_reviews
          FROM reviews AS rev;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

Количество уникальных идентификаторов в таблице reviews


,cnt_unique_book_id_in_reviews
0,994


In [231]:
print('Количество уникальных обзоров в таблице reviews с самым большим числом обзоров')
query = '''SELECT DISTINCT(rev.book_id) as book_id,
                COUNT(DISTINCT(rev.review_id)) as cnt_reviews
          FROM reviews AS rev
          GROUP BY rev.book_id
          ORDER BY cnt_reviews DESC
          LIMIT 1;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

Количество уникальных обзоров в таблице reviews с самым большим числом обзоров


,book_id,cnt_reviews
0,948,7


**Задача 3:** Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [232]:
query = '''SELECT DISTINCT(p.publisher) as publisher,
                  COUNT(b.book_id) as books_cnt
          FROM books AS b
          JOIN publishers AS p ON b.publisher_id = p.publisher_id
          WHERE b.num_pages > 50
          GROUP BY p.publisher
          ORDER BY COUNT(b.book_id) DESC
          LIMIT 1;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,books_cnt
0,Penguin Books,42


**Задача 4:** Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [234]:
query = '''SELECT DISTINCT(au.author) as author,
                  AVG(rat.rating) as avg_rating
          FROM authors AS au 
          JOIN books AS b ON au.author_id = b.author_id
          JOIN ratings AS rat ON b.book_id = rat.book_id
          WHERE b.book_id IN (SELECT DISTINCT(book_id) as book_id
                              FROM ratings as rat
                              GROUP BY book_id
                              HAVING COUNT(DISTINCT(rat.rating_id)) >= 50)
          GROUP BY au.author
          ORDER BY AVG(rat.rating) DESC
          LIMIT 1;'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


**Задача 5:** Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [235]:
query = '''SELECT ROUND(AVG(rev.cnt)) AS avg_cnt
          FROM (SELECT reviews.username,
                      COUNT(review_id) AS cnt
                FROM reviews
                GROUP BY reviews.username) as rev
          WHERE username in (SELECT rat.username
                              FROM ratings as rat
                              GROUP BY rat.username
                              HAVING COUNT(DISTINCT(rat.rating_id)) > 48);'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_cnt
0,24.0


## Вывод:  
1. После 1 января 2000 года вышло 819 книг.  
2. В БД содержатся данные о 1000 изданиях. Количество ревью и средние рейтинги могут быть самыми разными. При этом количество наименований 999, а количество ревью 944, т.е. для некоторых книг обзоры отсутствуют.    
3. Издательство выпустившее наибольшее количество книг не считая брошюр - Penguin Books (42 книги).
4. Автор с наиболее высокой средней оценкой из тех что были оценены 50 и более раз: J.K. Rowling/Mary GrandPré, средний рейтинг 4.28.    
5. Среднее количество обзоров от пользователей, среди поставивших больше 48 оценок - 24.